# Classifying adversarial images

In order to aid in the correct classification of traffic signs, we wanted to explore designing a binary classifier which could distinguish between adversarial images and untampered images. In this notebook, the class label 1 represents an adversarial image and the class label 0 represents an untampered image.

The idea would be to run images through this classifier first, before inputting the images into the multiclass CNN. The output from this binary classifier would serve as an additional feature for the multiclass CNN and would advise on whether the image it was classifying has been perturbed. 

First, import the necessary packages.

In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
np.random.seed(42)

from matplotlib import style
style.use('fivethirtyeight')

Next, define the file paths for the adversarial images and real images. If you would like to replicate this notebook, the adversarial images can be found [here](https://drive.google.com/drive/u/0/folders/1B7XcUAJHHn1rkReNK7sENGZ61y_A8OY8). The untampered image dataset can be found [here](https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign). Once downloaded, you can edit the file paths below to match your desktop file paths.

In [3]:
#untampered dataset, stored as image files within folders
data_dir = "C:\\Users\\Team Knowhow\\Documents\\YEAR 4\\Data Science Toolbox\\DST-NN-Project\\Erin Pollard"
untampered_train_path = "C:\\Users\\Team Knowhow\\Documents\\YEAR 4\\Data Science Toolbox\\DST-NN-Project\\Erin Pollard\\Train"

#adversarial image dataset, stored as 4 npy files - 4 types of adversarial attacks
adv_data1 = np.load('noisy_training-004.npy')
adv_data2 = np.load('black_patched_training-001.npy')
adv_data3 = np.load('white_patched_training-002.npy')
adv_data4 = np.load('same_image_patched_training-003.npy')

Rather than 4 separate arrays, we would like just one array containing the adversarial images.

In [7]:
adv_total = np.concatenate((adv_data1,adv_data2,adv_data3,adv_data4),axis=0)

MemoryError: Unable to allocate 8.74 GiB for an array with shape (381840, 32, 32, 3) and data type float64